## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2021/seasontype/2

In [1]:
SET_WEEK = 7
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_7


In [2]:
from scrapers.scrape_2021 import get_2021_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2021 data

In [4]:
%%time

X = get_2021_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game', 'offense_passing_AVG', 'offense_passing_YDS/G', 'offense_passing_RTG', 
             'offense_receiving_AVG', 'offense_rushing_YDS/G', 'defense_points_per_game', 'defense_passing_AVG', 
             'defense_receiving_AVG', 'defense_rushing_YDS/G', 'defense_passing_SYL_per_game', 
             'offense_downs_Third Downs_ATT_per_game', 'defense_downs_Third Downs_ATT_per_game', 
             'defense_downs_Fourth Downs_ATT_per_game', 'defense_pass_to_rush_ratio', 
             'defense_downs_First Downs_penalty_ratio', 'offense_passing_TD_per_game', 'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 18.7 s


,Team,games_played,season,offense_points_per_game,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,offense_rushing_YDS/G,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_Fourth Downs_ATT_per_game,defense_pass_to_rush_ratio,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,offense_pass_TD_to_INT
0,Dallas Cowboys,6,2021,34.2,8.5,296.5,115.3,11.5,164.3,24.3,...,12.4,86.2,16.166667,12.5,11.666667,1.500000,1.892562,0.090909,2.666667,4.0
1,Tampa Bay Buccaneers,7,2021,33.3,7.5,324.3,108.2,11.2,99.1,21.0,...,9.8,67.4,12.428571,13.0,12.000000,0.857143,2.228346,0.113475,3.000000,7.0


In [5]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [6]:
X['games_played'].unique().tolist()

[6, 7]

In [8]:
# X[X['games_played'] == 6]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 4

predictions.sort_values('pred_mean', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
6,Arizona Cardinals,2021,week_7,0.300602,0.352088,0.475791,0.412593,0.385269
5,Buffalo Bills,2021,week_7,0.285716,0.246341,0.411262,0.381045,0.331091
7,Los Angeles Rams,2021,week_7,0.315021,0.322383,0.277161,0.323202,0.309442


In [10]:
predictions.to_csv('weekly_runs/' + week_name + '.csv', index = False)

### Check weekly run

In [11]:
predictions

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
6,Arizona Cardinals,2021,week_7,0.300602,0.352088,0.475791,0.412593,0.385269
5,Buffalo Bills,2021,week_7,0.285716,0.246341,0.411262,0.381045,0.331091
7,Los Angeles Rams,2021,week_7,0.315021,0.322383,0.277161,0.323202,0.309442
0,Dallas Cowboys,2021,week_7,0.273340,0.230925,0.331979,0.286301,0.280636
12,Cincinnati Bengals,2021,week_7,0.209799,0.149072,0.421751,0.277438,0.264515
1,Tampa Bay Buccaneers,2021,week_7,0.245912,0.203894,0.307255,0.282241,0.259826
9,Las Vegas Raiders,2021,week_7,0.218720,0.175781,0.284814,0.254564,0.233470
8,Cleveland Browns,2021,week_7,0.128123,0.073338,0.147321,0.201126,0.137477
22,Green Bay Packers,2021,week_7,0.144716,0.079422,0.107943,0.214732,0.136703
28,New Orleans Saints,2021,week_7,0.117681,0.080497,0.113733,0.228486,0.135099
